## Análisis del laboratorio de masa

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import custom libraries
import warnings
import sys
import os

# Subir dos niveles desde notebooks/mod_tecnico/ hasta la raíz del repo
repo_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))  
sys.path.append(repo_path)

# Ahora intenta importar
from src.data_processing import ready_to_import

In [3]:
#Importar librerias para graficar
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
from plotly.subplots  import make_subplots

In [4]:
# Importamos lo necesario para trabajar con consultas SQL

from pandasql import sqldf

# Definir función de consulta
pysqldf = lambda q: sqldf(q, globals())

#### Se carga el dataset 

In [5]:
# Import data
# Import data from Excel files
url = "https://raw.githubusercontent.com/MiguelOscar-Insco/data-INSCO-hub/main/data/raw/masa_25.xlsx"
masa = pd.read_excel(url)


In [6]:
# Resumen estadístico de la tabla
masa.describe()

,lab_received_date,scheduled_delivery_date,cleaning_date,calibration_date,delivery_date,delivery_time,assigned_time,piece_count
count,462,455,475,476,481,481.000000,481.000000,481.000000
mean,2025-01-30 01:21:02.337662464,2025-02-18 17:56:02.637362432,2025-01-31 01:15:47.368421120,2025-02-09 17:29:44.873949440,2025-02-15 03:11:36.049895936,8.627859,12.295218,5.081081
min,2024-11-06 00:00:00,2024-12-19 00:00:00,2024-03-12 00:00:00,2024-03-13 00:00:00,2025-01-02 00:00:00,0.000000,0.000000,1.000000
25%,2025-01-14 00:00:00,2025-01-31 00:00:00,2025-01-13 12:00:00,2025-01-23 00:00:00,2025-01-24 00:00:00,5.000000,11.000000,1.000000
50%,2025-01-28 00:00:00,2025-02-18 00:00:00,2025-01-29 00:00:00,2025-02-10 00:00:00,2025-02-11 00:00:00,8.000000,14.000000,1.000000
75%,2025-02-25 00:00:00,2025-03-13 00:00:00,2025-02-28 00:00:00,2025-03-07 18:00:00,2025-03-11 00:00:00,11.000000,15.000000,4.000000
max,2025-03-27 00:00:00,2025-04-04 00:00:00,2025-03-28 00:00:00,2025-03-31 00:00:00,2025-03-31 00:00:00,44.000000,30.000000,58.000000
std,NaN,NaN,NaN,NaN,NaN,5.915796,5.237302,8.480350


In [7]:
masa.dtypes

report_number                      object
so                                 object
assignee                           object
calibrator                         object
supervisor                         object
approver                           object
classification                     object
lab_received_date          datetime64[ns]
scheduled_delivery_date    datetime64[ns]
cleaning_date              datetime64[ns]
calibration_date           datetime64[ns]
delivery_date              datetime64[ns]
delivery_time                     float64
process_status                     object
assigned_time                     float64
service_location                   object
substitution_reason                object
lab_observations                   object
priority                           object
piece_count                       float64
dtype: object

In [8]:
# Seleccionamos las columnas que contienen fechas y las convertimos a tipo datetime
col_fechas = [col for col in masa.columns if 'date' in col.lower() or 'fecha' in col.lower()]

for col in col_fechas:
    masa[col] = pd.to_datetime(masa[col], errors='coerce', format='%Y/%m/%d')

In [9]:
masa.isnull().sum()

report_number              125
so                         106
assignee                     0
calibrator                 106
supervisor                 106
approver                   106
classification             106
lab_received_date          125
scheduled_delivery_date    132
cleaning_date              112
calibration_date           111
delivery_date              106
delivery_time              106
process_status             106
assigned_time              106
service_location           106
substitution_reason        562
lab_observations           526
priority                   106
piece_count                106
dtype: int64

In [10]:
# Se hace una copia de la tabla para evitar problemas de referencia
mass = masa.copy()

**Se eliminan las filas donde 'report_number' es NaN y se une el contenido de 'assignee' de las filas con 'report_number' NaN a la fila anterior**

In [11]:
# Esta función es interna y creada para el proceso en el que se va a usar
mass = ready_to_import.to_prepare_db(mass)
# Verificamos nuevamente los tipos de datos de cada columna y la cantidad de datos nulos
mass.isnull().sum()

report_number                0
so                           0
assignee                     0
calibrator                   0
supervisor                   0
approver                     0
classification               0
lab_received_date            0
scheduled_delivery_date      7
cleaning_date                6
calibration_date             5
delivery_date                0
delivery_time                0
process_status               0
assigned_time                0
service_location             0
substitution_reason        437
lab_observations           401
priority                     0
piece_count                  0
dtype: int64

## Agregamos columnas temporales segmentadas

In [12]:
# Crear columnas derivadas de la fecha de recepción
mass['year_received'] = mass['lab_received_date'].dt.year
mass['month_received'] = mass['lab_received_date'].dt.month
mass['week_received'] = mass['lab_received_date'].dt.isocalendar().week

# Crear columnas derivadas de la fecha de entrega
mass['year_delivery'] = mass['delivery_date'].dt.year
mass['month_delivery'] = mass['delivery_date'].dt.month
mass['week_delivery'] = mass['delivery_date'].dt.isocalendar().week

### Comparamos la cantidad de calibraciones con los mantenimientos

In [13]:
# Clasificar por tipo de servicio
mass['service_type'] = np.where(
    mass['report_number'].str.startswith('RSM', na=False),
    'Mantenimiento',
    np.where(
        mass['report_number'].str.startswith('CCM', na=False),
        'Calibración',
        'Otros'  # Valor por defecto si ninguna condición se cumple
    )
)

#### Mostramos un dataset con los tipos de servicio

In [14]:
service_type = mass['service_type'].value_counts()
service_type

service_type
Calibración      388
Mantenimiento     74
Name: count, dtype: int64

#### Mostramos un gráfico de barras con los tipos de servicio

In [38]:
import plotly.graph_objects as go

fig = go.Figure(data=[ 
    go.Pie(
        labels=service_type.index, 
        values=service_type.values, 
        hole=0.3, 
        textinfo='percent',  # Muestra el porcentaje y el nombre
        hoverinfo='label+value',  # Muestra nombre y porcentaje al pasar el mouse
        marker=dict(
            colors=['#66b3ff', '#99ff99', '#ff6666', '#ffcc99'],  # Puedes ajustar estos colores
        ),
        #pull=[0.1, 0, 0, 0],  # Hace "salir" ligeramente la primera porción (opcional)
    )
])

fig.update_layout(
    template="plotly+seaborn",  # Combina los templates de plotly y seaborn
    width=600,
    height=500,
    title='Distribución de Tipos de Servicios',
    title_x=0.5,
    title_font=dict(size=20, family='Verdana, sans-serif'),
    legend_title_text="Servicios",
    legend=dict(
        x=0.8, y=1.1,
        bgcolor="rgba(255,255,255,0.5)",
        bordercolor="Black",
        borderwidth=1
    ),
    showlegend=True,
)

fig.show()


#### La gráfica de barras muestra que la mayoría de los servicios son calibraciones (387), en comparación con los mantenimientos, que son solo 74. 

#### Añadir una columna para clasificar los equipos por tipos

In [16]:
classification_map = {
    'I': 'Balanza', 'II': 'Balanza', 'III': 'Balanza', 'IIII': 'Balanza',
    'E1': 'Pesa', 'E2': 'Pesa', 'F1': 'Pesa', 'F2': 'Pesa',
    'M1': 'Pesa', 'M2': 'Pesa', 'M3': 'Pesa',
    '1': 'Pesa', '2': 'Pesa', '3': 'Pesa', '4': 'Pesa',
    '5': 'Pesa', '6': 'Pesa',
    'M': 'NBS', 'S': 'NBS', 'S-1': 'NBS', 'P': 'NBS', 'Q': 'NBS',
    'T': 'NBS', 'F': 'NBS',
    'ONN': 'ONN'
}

# Asignar valores usando map()
mass['equipment_type'] = mass['classification'].map(classification_map).fillna('Otros')

## Análisis de los matentimientos

In [17]:
# Obtenemos los reportes de mantenimiento
mass_maintenance = mass[mass['report_number'].str.startswith('RSM', na=False)]
mass_maintenance.head(10)

,report_number,so,assignee,calibrator,supervisor,approver,classification,lab_received_date,scheduled_delivery_date,cleaning_date,...,priority,piece_count,year_received,month_received,week_received,year_delivery,month_delivery,week_delivery,service_type,equipment_type
388,RSM0001.M/2025,H2403,Guillermo Guzmán López,Guillermo Guzmán López,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2024-12-09,2025-01-06,2025-01-02,...,ORDINARIO,4.0,2024,12,50,2025,1,2,Mantenimiento,Pesa
389,RSM0002.M/2025,H2403,Guillermo Guzmán López,Guillermo Guzmán López,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2024-12-09,2025-01-06,2025-01-02,...,ORDINARIO,3.0,2024,12,50,2025,1,2,Mantenimiento,Pesa
390,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
391,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
392,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
393,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
394,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
395,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
396,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
397,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa


In [18]:
loc_eq = (mass_maintenance.groupby(['service_location', 'equipment_type'])['report_number']
          .count()
          .unstack()
          .fillna(0)
          .sort_values('Pesa', ascending=False)
          .rename(
            columns={
                'Pesa': 'Pesa',
                'Balanza': 'Balanza'
            }
)
        )
if 'index' not in loc_eq.columns:
  loc_eq.reset_index(inplace=True)
  loc_eq.rename(columns={'service_location': 'Ubicación'}, inplace=True)
loc_eq

equipment_type,Ubicación,Balanza,Pesa
0,LAB,0.0,66.0
1,SITIO,8.0,0.0


In [19]:
# Contamos la cantidad de reportes por tipo de equipo
maintenance_per_type = mass_maintenance.groupby('equipment_type')['report_number'].count().sort_values(ascending=False)
maintenance_per_type

equipment_type
Pesa       66
Balanza     8
Name: report_number, dtype: int64

### Gráfico de barras con los servicios de mantenimiento clasificados por tipo

In [20]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Crea el subplot: 1 fila, 2 columnas
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.3, 0.7],
    specs=[[{"type": "table"}, {"type": "domain"}]]
)

# Añade la tabla
fig.add_trace(
    go.Table(
        header=dict(
            values=loc_eq.columns,
            fill_color='paleturquoise',
            align='center'
        ),
        cells=dict(
            values=loc_eq.values.T,
            fill_color='lavender',
            align='center'
        )
    ),
    row=1, col=1
)

# Añade el gráfico de pastel
fig.add_trace(
    go.Pie(
        labels=maintenance_per_type.index,
        values=maintenance_per_type.values,
        hole=0.3,
        textinfo='percent',
        hoverinfo='label+value',
        marker=dict(colors=['#66b3ff', '#99ff99', '#ff6666', '#ffcc99']),
    ),
    row=1, col=2
)

# Ajustes finales
fig.update_layout(
    title_text="Clasificación de mantenimientos por tipo de equipo",
    height=500,
    width=1000,
    template="seaborn"
)

fig.show()


**El gráfico muestra que en el primer trimestre del año el laboratorio le ha dado mantenimiento a 66 pesas y 8 balanzas.**

## Cantidad de mantenimientos por meses y tipos de equipos

In [21]:
# Contamos la cantidad de reportes por clasificación
maint_ext = mass_maintenance[['report_number', 'classification','assignee', 'delivery_date', 'assigned_time', 'delivery_time', 'piece_count', 'equipment_type', 'week_delivery']].copy()
maint_ext['month_deliv_name'] = maint_ext['delivery_date'].dt.month_name(locale='es')


# Contamos la cantidad de reportes por mes y tipo de equipo
maint_month_type = (maint_ext.groupby(['month_deliv_name', 'equipment_type'])['piece_count']
        .sum()
        .unstack()
        .fillna(0)
        #.sort_values(('piece_count', 'Pesa'), ascending=False) # Corrección en la columna de ordenamiento
        .rename(
            columns={
                'Pesa': 'Pesa',
                'Balanza': 'Balanza'
            }
        )
        )

if 'index' not in maint_month_type.columns:
  maint_month_type.reset_index(inplace=True)
  maint_month_type.rename(columns={'month_deliv_name': 'Mes'}, inplace=True)

maint_mt_melted = maint_month_type.melt(id_vars=['Mes'], var_name='Equipo', value_name='Total').sort_values('Total', ascending=False)

In [22]:
# Crea el subplot: 1 fila, 2 columnas
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.3, 0.8],
    specs=[[{"type": "table"}, {"type": "bar"}]],
    horizontal_spacing=0.1  # Espaciado entre tabla y gráfico
)

# Añade la tabla
fig.add_trace(
    go.Table(
        header=dict(
            values=maint_month_type.columns,
            fill_color='paleturquoise',
            align='center',
            line_color='darkslategray'
        ),
        cells=dict(
            values=maint_month_type.values.T,
            fill_color='lavender',
            align='center',
            line_color='darkslategray'
        )
    ),
    row=1, col=1
)

# Configuración del gráfico de barras apiladas
color_sequence = ['#66b3ff', '#90fa99']
month_order = ['Enero', 'Febrero', 'Marzo']

# Añadir cada equipo como barra apilada
for i, equipo in enumerate(maint_mt_melted['Equipo'].unique()):
    subset = maint_mt_melted[maint_mt_melted['Equipo'] == equipo]
    
    # Filtrar texto solo para valores mayores a 0
    texto_barras = np.where(subset['Total'] > 0, 
                          subset['Total'].round().astype(int).astype(str), 
                          '')
    
    fig.add_trace(go.Bar(
        x=subset['Mes'],
        y=subset['Total'],
        name=equipo,
        marker_color=color_sequence[i % len(color_sequence)],
        text=texto_barras,  # Usamos el texto filtrado
        textposition='outside',
        texttemplate='<b>%{text}</b>',  # Texto en negrita
        textfont=dict(size=12, color='black'),
        hovertemplate=(
            '<b>Tipo:</b> %{fullData.name}<br>'
            '<b>Mes:</b> %{x}<br>'
            '<b>Cantidad:</b> %{y}<extra></extra>'
        ),
        hoverlabel=dict(
            bgcolor='white',
            font_size=12,
            font_family="Arial"
        )
    ), row=1, col=2)

# Resto del código permanece igual...

# Configurar layout para barras apiladas
fig.update_layout(
    title="Mantenimientos totales a piezas por meses y tipos de equipos",
    xaxis_title="Mes",
    yaxis_title="Número de mantenimientos",
    legend_title="Tipo de Equipo",
    template="plotly",
    barmode='stack',
    margin=dict(l=20, r=20, t=50, b=20),
    showlegend=True,
    uniformtext_minsize=10  # Tamaño mínimo para el texto
)

# Ordenar los meses y ajustar eje X
fig.update_xaxes(
    categoryorder='array',
    categoryarray=month_order,
    row=1, col=2
)

fig.update_yaxes(
    rangemode='tozero',
    row=1, col=2
)

fig.show()

**Análisis Descriptivo:**
* **Enero:** Se observa una cantidad significativamente mayor de mantenimientos en Enero (167 piezas en total). Este mes está compuesto únicamente por mantenimientos realizados en equipos tipo "Pesa".
* **Febrero:** La cantidad total de mantenimientos disminuye drásticamente en Febrero, con un total de 16 mantenimientos. De estos, 14 corresponden a equipos tipo "Pesa" y 2 a equipos tipo "Balanza".
* **Marzo:** El número total de mantenimientos aumenta en Marzo a 39. Este mes presenta una distribución con 33 mantenimientos realizados en equipos tipo "Pesa" y 6 en equipos tipo "Balanza".

### Agrupamos los mantenimientos por su clasificación

In [23]:
# Cantidad de mantenimientos por clasificación
maintenance_class = mass_maintenance.groupby('classification')['report_number'].count().sort_values(ascending=False)
maintenance_class

classification
M1     61
M2      5
I       4
II      3
III     1
Name: report_number, dtype: int64

### Gráfico de barras con los servicios de mantenimiento agrupados por clasificación

In [24]:
colors = ['#66b3ff'] * 5
colors[0] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=maintenance_class.index,
    y=maintenance_class.values,
    marker_color=colors,
    text=maintenance_class.values,
    textposition='outside',
    name='Número de Mantenimientos' # Añade un nombre para la leyenda
)])

fig.update_layout(
    title_text='Informes de mantenimiento por clasificación',
    width=800,  # Cambia el ancho del gráfico
    height=600, # Cambia la altura del gráfico
    xaxis_title= 'Clasificación',
    yaxis_title= 'Cantidad',
    legend_title_text='Variable', # Título de la leyenda
    legend=dict(
        orientation="h",      # Orientación horizontal de la leyenda
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

- El gráfico muestra que las pesas del tipo *M1* son las que más mantenimiento han recibido, con un total de 61 servicios. 
- Las balanzas del tipo *III* son las que menos mantenimiento han recibido, con solo 1 servicio.

## Analizamos los mantenimientos teniendo en cuenta su clasificación y la comparativa con sus tiempos asignados y de entrega.

In [25]:
maintenance_time = (mass_maintenance.groupby('classification')[['delivery_time', 'assigned_time', 'piece_count']]
                                               .mean().round(2)
                                               #.unstack(fill_value=0)
                                               #.reset_index()
                                               .sort_values('assigned_time', ascending=False)
                                               )  

'''maintenance_time = maintenance_time.rename(
    columns={
        'Tiempo_Entrega': 'delivery_time' ,
        'Tiempo_Asignación':'assigned_time',
        'Cantidad_Piezas':'piece_count'
    }
)'''
maintenance_time

,delivery_time,assigned_time,piece_count
classification,,,
M1,8.38,14.66,2.46
M2,8.00,9.60,12.80
III,2.00,5.00,1.00
I,3.00,3.75,1.00
II,3.00,3.33,1.00


In [26]:
from plotly.subplots import make_subplots

df = maintenance_time.reset_index()

# Configurar figura y ejes
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Barras agrupadas para Tiempo de Entrega
fig.add_trace(go.Bar(
    x=df['classification'],
    y=df['delivery_time'],
    name='Tiempo de Entrega',
    marker_color='#66b3ff',
    text=[f'{val:.1f}' for val in df['delivery_time']],
    textposition='outside',
    textfont=dict(color='black')
), secondary_y=False)

# Barras agrupadas para Tiempo Asignado
fig.add_trace(go.Bar(
    x=df['classification'],
    y=df['assigned_time'],
    name='Tiempo Asignado',
    marker_color='#99ff99',
    text=[f'{val:.1f}' for val in df['assigned_time']],
    textposition='outside',
    textfont=dict(color='black')
), secondary_y=False)

# Línea para Promedio de Piezas (segundo eje Y)
fig.add_trace(go.Scatter(
    x=df['classification'],
    y=df['piece_count'],
    mode='markers+lines+text',
    marker=dict(color='black', symbol='circle'),
    line=dict(color='black', dash='dash'),
    name='Promedio de Piezas',
    text=[f'{val:.1f}' for val in df['piece_count']],
    textposition='top center',
    textfont=dict(color='black')
), secondary_y=True)

# Configurar etiquetas de los ejes
fig.update_xaxes(title_text="Clasificación del Equipo")
fig.update_yaxes(title_text="Tiempo Promedio", secondary_y=False)
fig.update_yaxes(title_text="Promedio de Piezas", secondary_y=True)

# Configurar el título
fig.update_layout(
    template='plotly',
    title_text="Tiempo de entrega vs Tiempo asignado en los servicios de Mantenimiento",
    barmode='group',  # Asegura que las barras se agrupen
    width=1000,
    height=800,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

# **Conclusiones del Análisis del Gráfico**

## ✅ **Eficiencia en los tiempos de entrega**  
- En la mayoría de los casos, los tiempos de entrega son mucho menores a los asignados.  
- Esto sugiere que la planificación es conservadora.  
- Se podrían ajustar los tiempos asignados para reflejar mejor la realidad y mejorar la eficiencia.

## ✅ **Mayor complejidad en M1 y M2**  
- Estas clasificaciones tienen más piezas y mayores tiempos de entrega.  
- Esto indica que los equipos en estas categorías pueden ser más complejos al momento de darles mantenimiento.  
- La diferencia significativa en piezas con respecto a las otras categorías puede justificar el mayor tiempo de entrega.

## ✅ **Optimización en la planificación**  
- Dado que el tiempo asignado es mayor al tiempo de entrega en la mayoría de los casos, se podrían optimizar los recursos.  
- Ajustar los tiempos asignados evitaría subutilización de recursos y permitiría una planificación más precisa.  

## 💡 **Próximo paso**  
- Analizar si existe una relación directa entre el número de piezas utilizadas y el tiempo de entrega.  
- Determinar si este factor es clave en la planificación del mantenimiento.  


### Gráfico de barras con los servicios de mantenimiento y cantidad de piezas agrupados por clasificación

In [27]:
maint = pysqldf("""
SELECT classification, COUNT(report_number) AS rep_qty, SUM(piece_count) AS total_pieces, AVG(delivery_time) AS deliv_time_avg
FROM mass_maintenance
GROUP BY classification
ORDER BY rep_qty DESC
""")
maint

,classification,rep_qty,total_pieces,deliv_time_avg
0,M1,61,150.0,8.377049
1,M2,5,64.0,8.000000
2,I,4,4.0,3.000000
3,II,3,3.0,3.000000
4,III,1,1.0,2.000000


In [28]:
import math

df = maint.reset_index()

# Crear figura con eje secundario
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Barras para cantidad de reportes
fig.add_trace(
    go.Bar(
        x=df['classification'],
        y=df['rep_qty'],
        name='Cantidad de reportes',
        marker_color='#66b3ff',
        text=[f'{val:.1f}' for val in df['rep_qty']],
        textposition='outside',
        width=0.4,
        offset=-0.2,  # Desplaza las barras hacia la izquierda
        hovertemplate=  # Plantilla personalizada
            '<b>Clasificación:</b> %{x}<br>' +
            '<b>Cantidad:</b> %{y:.0f} informes<extra></extra>',
        hoverlabel=dict(
            bgcolor='white',
            font_size=12,
            font_family="Arial",
            font_color='black'
        )
    ),
    secondary_y=False
)

# Barras para cantidad de piezas
fig.add_trace(
    go.Bar(
        x=df['classification'],
        y=df['total_pieces'],
        name='Cantidad de piezas',
        marker_color='#99ff99',
        text=[f'{val:.0f}' for val in df['total_pieces']],  # Enteros sin decimales
        textposition='outside',
        width=0.4,
        offset=0.2,
        hovertemplate=  # Plantilla personalizada
            '<b>Clasificación:</b> %{x}<br>' +
            '<b>Cantidad:</b> %{y:.0f} piezas<extra></extra>',
        hoverlabel=dict(
            bgcolor='white',
            font_size=12,
            font_family="Arial",
            font_color='black'
        )
    ),
    secondary_y=False
)

# Línea para tiempo de entrega promedio
fig.add_trace(
    go.Scatter(
        x=df['classification'],
        y=df['deliv_time_avg'],
        mode='lines+markers+text',
        name='Tiempo de entrega promedio',
        line=dict(color='black', dash='dash'),
        marker=dict(symbol='circle', size=10),
        text=[f'{val:.1f}' for val in df['deliv_time_avg']],
        textposition='top center',
        hovertemplate=  # Plantilla personalizada
            '<b>Clasificación:</b> %{x}<br>' +
            '<b>Tiempo promedio:</b> %{y:.1f} días<extra></extra>',
        hoverlabel=dict(
            bgcolor='white',
            font_size=12,
            font_family="Arial",
            font_color='black'
        )
    ),
    secondary_y=True
)

# Agregar anotaciones para las razones (piezas/reportes) > 1
annotations = []
for i, row in df.iterrows():
    if row['rep_qty'] != 0:
        ratio = row['total_pieces'] / row['rep_qty']
        
        if ratio > 1:
            # Calcular posición de la anotación a 75°
            offset = 5
            angle = math.radians(75)
            dx = offset * math.cos(angle) + 30
            dy = offset * math.sin(angle) 
            
            y_base = min(row['rep_qty'], row['total_pieces'])
            
            annotations.append(dict(
                x=row['classification'],
                y=y_base,
                xshift=dx,
                yshift=dy,
                text=f'Razón: {ratio:.2f}',
                showarrow=True,
                arrowhead=3,
                arrowsize=2,
                arrowwidth=1,
                arrowcolor='red',
                font=dict(color='red'),
                ax=20,
                ay=-30
            ))

# Configurar ejes y título
fig.update_layout(
    title='Comparación entre la cantidad de reportes y la cantidad de piezas en los mantenimientos',
    title_font=dict(size=20, family='Arial', color='black'),
    xaxis_title='Clasificación del Equipo',
    barmode='group',
    annotations=annotations,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=600,
    width=1200,
    margin=dict(t=100)
)

# Configurar títulos de los ejes Y
fig.update_yaxes(title_text="Cantidad de reportes y piezas", secondary_y=False)
fig.update_yaxes(title_text="Tiempo de entrega promedio (días)", secondary_y=True)

# Mostrar gráfico
fig.show()

# Análisis de Servicios de Mantenimiento

Este reporte presenta un análisis detallado de los servicios de mantenimiento en el laboratorio de masa, enfocándose en la cantidad de reportes, el número de piezas enviadas por los clientes y el tiempo de entrega de los servicios. En este contexto, **la cantidad de piezas se refiere al número de componentes que los clientes envían a la instalación para su mantenimiento/calibración**.

---

## 1. Resumen 

- **Categoría M1:**  
  - Registra el mayor número de servicios de mantenimiento (61 reportes).  
  - Acumula el mayor volumen de piezas enviadas (150 piezas), con un promedio de **2.46 piezas por servicio**.
  
- **Categoría M2:**  
  - Aunque presenta menos reportes en comparación con M1, cada servicio involucra un número significativamente mayor de piezas, con un promedio de **12.80 piezas por servicio**.
  
- **Tiempo de Entrega:**  
  - El tiempo de entrega promedio se mantiene similar entre las categorías M1 y M2, lo que indica que el volumen de piezas enviadas no afecta de forma significativa la eficiencia del proceso de mantenimiento.

---

## 2. Análisis Detallado

### 2.1 Cantidad de Reportes y Piezas
- **Reportes de Mantenimiento:**  
  La visualización evidencia que la **categoría M1** es la más enviada por los clientes, sugiriendo una alta demanda.  
  En contraste, la **categoría M2** y otras presentan volúmenes de reportes considerablemente menores.

- **Piezas Enviadas:**  
  - **M1:** Con un total de 150 piezas, el promedio es de 2.46 piezas por servicio, lo que indica que cada solicitud de mantenieminto implica un volumen moderado de piezas.  
  - **M2:** Destaca por requerir, en promedio, 12.80 piezas por servicio, lo que sugiere que, si bien se realizan menos servicios, cada uno involucra un mayor número de piezas enviadas por los clientes.

---

## 3. Interpretación y Conclusiones

- **Alta Demanda en M1:**  
  La elevada cantidad de reportes y piezas en la categoría M1 subraya una alta demanda y una operación con un alto volumen de actividad.  
  Sin embargo, el promedio moderado de piezas por servicio permite un procesamiento eficiente.

- **Volumen de Piezas en M2:**  
  Aunque M2 tiene menos servicios reportados, la alta cantidad de piezas por servicio indica que este tipo de pesas llega en volúmenes mayores.
  

---

## 4. Recomendaciones

- **Monitoreo Continuo:**  
  Se recomienda mantener un monitoreo constante de los indicadores de volumen y tiempo de entrega, para detectar oportunamente cualquier desviación en la eficiencia operativa.
  
- **Optimización de Recursos:**  
  Dado que el volumen de piezas no impacta el tiempo de entrega, se podría explorar la posibilidad de centralizar ciertos procesos para optimizar el manejo de piezas en categorías con altos promedios (como M2).
---



## **Analicemos con detenimiento el caso particular de las pesas M1**

In [29]:
M1 = mass_maintenance[mass_maintenance['classification'] == 'M1']
M1

,report_number,so,assignee,calibrator,supervisor,approver,classification,lab_received_date,scheduled_delivery_date,cleaning_date,...,priority,piece_count,year_received,month_received,week_received,year_delivery,month_delivery,week_delivery,service_type,equipment_type
388,RSM0001.M/2025,H2403,Guillermo Guzmán López,Guillermo Guzmán López,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2024-12-09,2025-01-06,2025-01-02,...,ORDINARIO,4.0,2024,12,50,2025,1,2,Mantenimiento,Pesa
389,RSM0002.M/2025,H2403,Guillermo Guzmán López,Guillermo Guzmán López,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2024-12-09,2025-01-06,2025-01-02,...,ORDINARIO,3.0,2024,12,50,2025,1,2,Mantenimiento,Pesa
390,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
391,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
392,RSM0003.M/2025,I0036,"Guillermo Guzmán López, Mario de Jesús García,...",Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-01-14,2025-02-05,2025-01-20,...,ORDINARIO,1.0,2025,1,3,2025,1,4,Mantenimiento,Pesa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,RSM0014.M/2025,I0360,Mario de Jesús García,Mario de Jesús García,Leobardo Maximiliano Brito Fonseca,Leobardo Maximiliano Brito Fonseca,M1,2025-03-06,2025-03-24,2025-03-06,...,ORDINARIO,10.0,2025,3,10,2025,3,12,Mantenimiento,Pesa
457,RSM0015.M/2025,I0361,Sergio Cruz Santiago,Sergio Cruz Santiago,Gabriel Gallardo Camacho,Gabriel Gallardo Camacho,M1,2025-03-06,2025-03-24,2025-03-12,...,ORDINARIO,5.0,2025,3,10,2025,3,12,Mantenimiento,Pesa
458,RSM0016.M/2025,I0361,Sergio Cruz Santiago,Sergio Cruz Santiago,Gabriel Gallardo Camacho,Gabriel Gallardo Camacho,M1,2025-03-06,2025-03-24,2025-03-12,...,ORDINARIO,5.0,2025,3,10,2025,3,12,Mantenimiento,Pesa
459,RSM0017.M/2025,I0361,Sergio Cruz Santiago,Sergio Cruz Santiago,Gabriel Gallardo Camacho,Gabriel Gallardo Camacho,M1,2025-03-06,2025-03-24,2025-03-12,...,ORDINARIO,5.0,2025,3,10,2025,3,12,Mantenimiento,Pesa


In [30]:
m1_2 = mass_maintenance.loc[mass_maintenance['classification'].isin(['M1', 'M2'])].sort_values('classification')

m1_2grp = m1_2.groupby('week_delivery', as_index=False).agg({
    'delivery_time': 'mean',
    'assigned_time': 'mean' 
})

m1_2grp

,week_delivery,delivery_time,assigned_time
0,2,21.333333,18.0
1,4,8.000000,15.0
2,5,5.000000,12.0
3,8,6.000000,12.0
4,9,0.000000,3.0
5,12,8.800000,11.0
6,14,10.000000,11.0


In [33]:
m1 = M1[['week_delivery', 'delivery_time', 'assigned_time']]
m1_grp = m1.groupby('week_delivery', as_index=False).agg({
    'delivery_time': 'mean',
    'assigned_time': 'mean' 
})

In [35]:
df = m1_grp

# Colores básicos pero fáciles de distinguir
custom_colors = ['#2ca02c', '#d62728']

# Crear un gráfico simple y directo
fig = px.line(df, x="week_delivery", y=df.columns[1:],
              color_discrete_sequence=custom_colors,
              title='Comparación: Tiempo de Mantenimiento vs Asignado de las pesas M1',
              labels={'value': 'Días', 'week': 'Semana', 'variable': 'Tipo'},
              markers=True
             )

# Hover básico y claro
fig.update_traces(
    hovertemplate="<br>Valor: %{y:.1f} días<extra></extra>",
    selector={'mode': 'lines+markers'}, # Aplica solo a las trazas de línea con marcadores
    line=dict(width=2),
    marker=dict(size=6)
)

# Para los marcadores añadidos individualmente (si los tienes)
fig.update_traces(
    hovertemplate="<br>Valor: %{y:.1f} días<extra></extra>",
    selector={'mode': 'markers'}, # Aplica solo a las trazas de marcador
    marker=dict(size=8) # Puedes ajustar el tamaño solo para los marcadores aquí
)

# Ejes limpios y legibles
fig.update_xaxes(
    title="<b>Semana</b>",
    tickangle=45,
    tickfont=dict(size=10),
    gridcolor='lightgray'
)

fig.update_yaxes(
    title="<b>Días</b>",
    tickfont=dict(size=10),
    gridcolor='lightgray'
)

# Layout simple
fig.update_layout(
    legend_title="<b>Indicadores</b>",
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.2,
        xanchor="center",
        x=0.5
    ),
    title=dict(
        text='<b>Comparación: Tiempo de Mantenimiento vs Asignado de las pesas M1</b>',
        x=0.5
    ),
    plot_bgcolor='white',
    margin=dict(t=80, b=100, l=80, r=40)
)

# Mostrar gráfico
fig.show()

### 💡Se recomienda hacer una revisión de los tiempos asignados para los manteniemientos de las pesas M1  teniendo en cuenta las fechas en las que se reciben los las mismas, pues se puede observar hay un gran número de pesas que se entregan en un tiempo menor a 2/3 de los días asignados, lo que sugiere que el tiempo asignado es excesivo.

# Realizar el análisis de las calibraciones

### Eliminamos todos los reportes que no son calibraciones

In [36]:
# Se eliminan las filas que no son calibraciones
mass_cal = mass[~mass['report_number'].str.startswith('RSM', na=False)]

In [37]:
# Analizamos la información de la tabla
mass_cal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 388 entries, 0 to 387
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   report_number            388 non-null    object        
 1   so                       388 non-null    object        
 2   assignee                 388 non-null    object        
 3   calibrator               388 non-null    object        
 4   supervisor               388 non-null    object        
 5   approver                 388 non-null    object        
 6   classification           388 non-null    object        
 7   lab_received_date        388 non-null    datetime64[ns]
 8   scheduled_delivery_date  383 non-null    datetime64[ns]
 9   cleaning_date            382 non-null    datetime64[ns]
 10  calibration_date         388 non-null    datetime64[ns]
 11  delivery_date            388 non-null    datetime64[ns]
 12  delivery_time            388 non-null    

# Análisis del DataFrame

**📌 Estructura General**  
- **387 registros** | **22 columnas** (5 fechas, 3 numéricas, 14 texto/object)  

**🔍 Hallazgos Clave**  
1. **Variables Temporales** (completas en su mayoría):  
   - `delivery_time` (tiempo de entrega) y `assigned_time` - **0 nulos**  
   - Fechas críticas (`lab_received_date`, `calibration_date`, `delivery_date`) - **100% completas**  
   - `scheduled_delivery_date` y `cleaning_date` - **5-6 nulos**  

2. **Columnas con Datos Faltantes**:  
   - `substitution_reason` (94% nulos) y `lab_observations` (87% nulos) - **Considerar eliminar o imputar**  

3. **Variables Categóricas Relevantes**:  
   - `classification`, `priority`, `service_type`, `equipment_type` - **0 nulos**  
   - Útiles para segmentación (ej: análisis por tipo de equipo o prioridad).  

**⚠️ Siguientes acciones**  
- Limpieza mínima: Revisar filas con `scheduled_delivery_date` nulo.  
- Enfoque analítico: **`delivery_time` vs categorías** (prioridad, tipo de servicio).  
- Columnas opcionales: `substitution_reason` y `lab_observations` aportan poco valor.  


# Análisis de los errores de sustitución antes de realizar el análisis de las calibraciones

In [ ]:
# Obtenemos la cantidad de reportes por cada razón de sustitución
subs_reason = mass_cal.groupby('substitution_reason')['report_number'].count().sort_values(ascending=False)
subs_reason


### Se comparan los distintos tipos de errores de sustitución

In [ ]:
# Calcular el total de sustituciones
total = mass_cal['report_number'].count()

# Crear figura
plt.figure(figsize=(10, 6))

# Gráfico de barras horizontales corregido
ax = sns.barplot(
    y=subs_reason.index, 
    x=subs_reason.values, 
    hue=subs_reason.index,  # Asigna la variable al hue
    palette='viridis', 
    legend=False  # Desactiva la leyenda redundante
)

# Etiquetas y título
plt.xlabel('Cantidad', fontsize=12, fontweight='bold')
plt.ylabel('Razones de sustitución', fontsize=12, fontweight='bold')
plt.title('Razones de sustitución por tipo de error en Masa', fontsize=14, fontweight='bold')

# Agregar anotaciones con valores y porcentajes
for i, v in enumerate(subs_reason.values):
    porcentaje = (v / total) * 100
    ax.text(v - 0.7, i, f"{v} ({porcentaje:.2f}%)", ha='left', va='center', fontsize=10, fontweight='bold')

# Mostrar gráfico
plt.show()



# Análisis de Sustituciones en Masa

**📊 Distribución de Causas**:
- **Principal motivo**: Solicitud del cliente (3.62% del total de informes de calibraciones (máximo))
- **Errores menores**: 
  - Área de atención al cliente (máx 0.26% del total de informes de calibraciones) 


1. **Prioridad de Acción**:
   - Enfocarse en **solicitudes de clientes** (frecuencia alta).


**📌 Recomendación**: **Las razones de sustitución por solicitudes del cliente puede indicar que falta claridad al momento de captar los requerimientos que el cliente necesita, por lo que se recomienda optimizar el proceso en cuestión.**

## Tipos de errores vs Clasificación de equipo

In [ ]:
# 1. Crear una copia del DataFrame para preservar los datos originales
df_temp = mass_cal.copy()

# 2. Identificar errores no clasificados (NaN)
df_temp['classification'] = df_temp['classification'].fillna('No clasificado')  # Renombrar NaN

# 3. Agrupar incluyendo la nueva categoría
errores_por_classification = (
    df_temp.groupby(['classification', 'substitution_reason'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
)

# 4. Calcular el TOTAL correcto (suma de razones)
columnas_errores = errores_por_classification.columns.difference(['classification'])
errores_por_classification['Total'] = errores_por_classification[columnas_errores].sum(axis=1)

# 5. Ordenar y mostrar
errores_por_classification = errores_por_classification.sort_values('Total', ascending=False)
errores_por_classification

### Se comparan los distintos tipos de errores de sustitución por clasificación de equipo

In [ ]:
# Configurar el estilo
plt.style.use('ggplot')

# Crear gráfico de barras apiladas
errores_por_classification = errores_por_classification.drop(columns='Total', errors='ignore')  # Eliminar la columna 'Total' para el gráfico
ax = errores_por_classification.set_index('classification').plot(
    kind='bar', 
    stacked=True,
    figsize=(14, 7),
    colormap='tab20',  # Paleta de colores para múltiples categorías
    edgecolor='black',
    linewidth=0.5
)

# Personalizar el gráfico
plt.title('Distribución de los motivos de sustitución de informes por Clasificación', fontsize=16, pad=20)
plt.xlabel('Clasificación', fontsize=12)
plt.ylabel('Cantidad de sustituciones', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)

# Añadir etiquetas de totales encima de cada barra completa
for i, classification in enumerate(errores_por_classification['classification']):
    total = errores_por_classification.set_index('classification').loc[classification].sum()
    ax.text(i, 
            total + (ax.get_ylim()[1] * 0.02),  # Pequeño margen arriba de la barra
            f'{int(total)}', 
            ha='center', 
            va='bottom',
            fontsize=10,
            fontweight='bold',
            color='black')

# Añadir etiquetas para cada segmento (opcional)
for rect in ax.patches:
    height = rect.get_height()
    if height > 0:  # Mostrar solo etiquetas para barras con valores
        ax.text(rect.get_x() + rect.get_width()/2, 
                rect.get_y() + height/2, 
                f'{int(height)}', 
                ha='center', 
                va='center',
                fontsize=8,
                color='black')

# Mejorar la leyenda
plt.legend(
    title='Tipos de Sustituciones',
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    fontsize=9,
    frameon=True
)

# Ajustar márgenes
plt.tight_layout()
plt.show()

## 📌 Hallazgo Principal
**Las pesas E1 son la clasificación más crítica**, representando:
- **13 sustituciones totales** (entre ellas 7 por solicitud del cliente y 3 por errores de laboratorio)
- Esto equivale al **56.52% del total de sustituciones** 

#### Eliminar las filas que contienen los informes que corresponden a las sustituciones por errores

In [ ]:
mass_sin_errores = df_temp.drop(df_temp[df_temp['substitution_reason'].notna()].index)

In [ ]:
mass_sin_errores.loc[mass_sin_errores['report_number'] == 'CCM0095.M/2025', 'piece_count'] = 95

In [ ]:
mass_sin_errores = mass_sin_errores.drop(columns=['substitution_reason'], errors='ignore')
mass_sin_errores.head(10)

In [ ]:
# Eliminamos las columnas que no son necesarias
mass_sin_errores = mass_sin_errores.drop(columns=['substitution_reason', 'service_type', 'lab_observations', 'process_status'], errors='ignore')

# Verificamos nuevamente los tipos de datos de cada columna y la cantidad de datos nulos
mass_sin_errores.info()

#### Comenzamos a trabajar sobre esta nueva tabla

In [ ]:
# Verificamos los valores nulos de la columna 'approver' en la tabla

resultado = pysqldf("""
    SELECT *
    FROM mass_sin_errores
    WHERE approver IS NULL """)

resultado

In [ ]:
#Cambiar el valor de la columna approver a 'Gabriel Gallardo Camacho' para el report_number específico
mass_sin_errores.loc[mass_sin_errores['report_number'].isin(['CCM0049.M/2025', 'CCM0050.M/2025']), 'approver'] = 'Gabriel Gallardo Camacho'

In [ ]:
# Verificamos los valores nulos de las columnas con valores faltantes en la tabla

resultado = pysqldf("""
    SELECT report_number, scheduled_delivery_date, cleaning_date, service_location
    FROM mass_sin_errores
    WHERE scheduled_delivery_date IS NULL or cleaning_date IS NULL """)

resultado

**Los valores faltantes son consistentes con los datos y por tanto se mantienen de esa forma**

##### Buscamos si quedaron algunos valores duplicados

In [ ]:
no_dup = mass_sin_errores[mass_sin_errores.duplicated(keep=False)]
no_dup

**Lo podemos considerar como un solo certificado con dos piezas distintas**

In [ ]:
mass_sin_errores = mass_sin_errores.drop_duplicates()

**Lo podemos considerar como un solo certificado con dos piezas distintas**

In [ ]:
mass_sin_errores.loc[mass_sin_errores['report_number'] == 'CCM0058.M/2025', 'piece_count'] = 2

### Análisis de la cantidad de reportes por clasificación de equipo

In [ ]:
qty_per_class = mass_sin_errores.groupby(['classification'])['report_number'].count().sort_values(ascending=False)
qty_per_class = qty_per_class.reset_index()
qty_per_class

In [ ]:
plt.figure(figsize=(12, 7))
sns.set_style("whitegrid")

# Calculamos los porcentajes
total = qty_per_class['report_number'].sum()
qty_per_class['percentage'] = (qty_per_class['report_number'] / total) * 100

ax = sns.barplot(
    x='classification',
    y='report_number',
    hue='classification',  # Added to address warning
    data=qty_per_class,
    palette='magma',
    edgecolor='black',
    linewidth=0.5,
    alpha=0.85,
    dodge=False,  # Prevents side-by-side bars when using hue
    legend=False  # Hides redundant legend
)

# Añadir etiquetas con valores absolutos y porcentajes
for i, bar in enumerate(ax.patches):
    height = bar.get_height()
    percentage = qty_per_class['percentage'].iloc[i]
    
    ax.text(
        x=bar.get_x() + bar.get_width() / 2,
        y=height + (0.02 * max(qty_per_class['report_number'])),  # Offset dinámico
        s=f"{height:.0f}\n({percentage:.1f}%)",
        ha="center",
        va="bottom",
        fontsize=10,
        color='black',
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1)
    )

# Mejorar el título y etiquetas
plt.title("Distribución de equipos por clasificación\nCon porcentajes y valores absolutos", 
          fontsize=15, pad=20, fontweight='bold')
plt.xlabel("Clasificación del equipo", fontsize=12, labelpad=10)
plt.ylabel("Cantidad de equipos", fontsize=12, labelpad=10)

# Rotación y formato de ejes
plt.xticks(rotation=45, ha='right', fontsize=11)
plt.yticks(fontsize=10)

'''# Línea horizontal de referencia
plt.axhline(y=total/len(qty_per_class), color='red', linestyle='--', alpha=0.5, 
            label=f'Promedio: {total/len(qty_per_class):.0f} equipos')
'''
# Cuadrícula y leyenda
plt.grid(axis='y', linestyle='--', alpha=0.5)
#plt.legend(loc='upper right', framealpha=0.9)


# Ajustar márgenes
plt.tight_layout()
plt.show()

### Distribución de Equipos por Clasificación  


### 🔍 Hallazgos Clave  
1. **Dominancia clara**:  
   - La clasificación **E2** concentra el **39.1% del total** (363 equipos)  
   - Las **4 primeras categorías** (E2, F1, M1 Y F2) representan **89.3%** del total (363 equipos) 

2. **Cola larga**:  
   - 7 clasificaciones con ≤3.0% de participación cada una  
   - En conjunto suman solo **7.7%** del total


### Análisis de la cantidad de reportes por el tipo de equipo

In [ ]:
res = pysqldf("""
    SELECT equipment_type, COUNT(report_number) AS total_reportes
    FROM mass_sin_errores
    GROUP BY equipment_type """)
res

In [ ]:
labels = ['Pesas', 'Básculas']
sizes = [339, 24]
colors = ['#fa9999','#6ab3ff']
explode = (0.1, 0)  # Destacar pesas

fig, ax = plt.subplots(figsize=(8,6))
wedges, texts, autotexts = ax.pie(
    sizes, 
    explode=explode, 
    colors=colors,
    autopct='%1.1f%%',
    shadow=True, 
    startangle=90,
    textprops={'fontsize': 10}
)

# Configuración profesional de leyenda
ax.legend(
    wedges,
    labels,
    title="Tipos de Equipo",
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1),
    frameon=True,
    shadow=True
)

# Añadir título
plt.title('Reportes por Tipo de Equipo\n(Total: 363 reportes)', 
          fontweight='bold', pad=20)

# Ajustar layout para evitar corte de leyenda
plt.tight_layout()
plt.show()

## 📊 Distribución Global
- **Total reportes analizados**: 363 (100%)
- **Dominancia clara**:
  - **Pesas**: 339 reportes (93.4%)
  - **Básculas**: 24 reportes (6.6%)
#### 📊 Se muestra una dominancia clara de las pesas sobre las básculas, con un 93.4% de los reportes analizados.

## Análisis de los tiempos de entrega por clasificación de equipo

In [ ]:
class_del_time = mass_sin_errores.groupby(['classification'])['delivery_time'].mean().round(2).reset_index()
class_del_time

In [ ]:

plt.figure(figsize=(12, 8))
# Sintaxis básica
ax = sns.barplot(
    x='classification',  # Variable categórica (eje x)
    y='delivery_time',   # Variable numérica (eje y)
    data=class_del_time,
    #estimator='mean',  # Función de agregación (mean, sum, median, etc.)
    palette='magma',            # Intervalo de confianza (o 'sd' para desviación estándar)
    #color='blue'       # Color de las barras
)

# Añadir los valores en las barras
for bar in ax.patches:
    height = bar.get_height()
    ax.text(
        x=bar.get_x() + bar.get_width() / 2,  # Posición horizontal centrada
        y=height + 0.2,                       # Posición vertical (altura + offset)
        s=f"{height:.2f}",                    # Texto (2 decimales)
        ha="center",                           # Alineación horizontal
        va="bottom",                           # Alineación vertical
        fontsize=10
    )

plt.title("Promedio de tiempo por clasificación de equipo", fontsize=14)
plt.xlabel("Clasificación", fontsize=12)
plt.ylabel("Tiempo promedio", fontsize=12)
plt.xticks(rotation=45, ha='left', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Solo líneas de cuadrícula en el eje y

plt.show()

# Análisis de Casos Atípicos y Recomendaciones

## 🔴 Casos Atípicos Identificados

### 1. Demora Extrema en Calibración (37.33 días)
- **Descripción**:  
  Tiempo de calibración 7.5 veces mayor que el segundo valor más alto (5 días)
- **Impacto**:  
  Puede retrasar toda la cadena de procesos del laboratorio

## 🛠 Recomendaciones Específicas

### Para Demora de 37.33 Días
1. **Acción Inmediata**:  
    - Revisar el caso específico para entender la causa de la demora.
    - Considerar la posibilidad de un mapeo del proceso que permita identificar los cuellos de botella.

### --------------------------------------------------------------------------------------------------

## Análisis de los tiempos de entrega vs los tiempos asignados

In [ ]:
# Calcular el promedio de tiempos de entrega y asignación por clasificación
tiempos = pysqldf('''
    SELECT classification, AVG(delivery_time), AVG(assigned_time)
    FROM mass_sin_errores
    GROUP BY classification
''')
tiempos

In [ ]:

# Configurar el estilo
equipos = tiempos['classification'].tolist()
asignado = tiempos['AVG(assigned_time)'].tolist()
real = tiempos['AVG(delivery_time)'].tolist()

plt.figure(figsize=(14, 6))
ax = sns.barplot(x=equipos, y=asignado, color='#4CAF00', label='Tiempo Asignado')
sns.barplot(x=equipos, y=real, color='#1191F3', label='Tiempo de Entrega Real', alpha=0.7)

# Personalización
plt.title('Comparación de Tiempos Asignados vs Reales por Clasificación', pad=20)
plt.xlabel('Clasificaciones')
plt.ylabel('Días')
plt.xticks(rotation=45)
plt.legend()

# Añadir valores
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                xytext=(0, 5), 
                textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
diferencias = [r - a for r, a in zip(real, asignado)]

plt.figure(figsize=(12, 6))
bars = plt.barh(equipos, diferencias, color=np.where(np.array(diferencias) > 0, '#e74c3c', '#2ecc71'))

# Personalización
plt.title('Diferencia entre Tiempo Real y Asignado', pad=15)
plt.xlabel('Diferencia (Real - Asignado) en Días')
plt.grid(axis='x', alpha=0.3)

# Añadir valores
for bar in bars:
    width = bar.get_width()
    plt.text(width/2, bar.get_y() + bar.get_height()/2, 
             f'{width:.1f}', 
             va='center', 
             color='white' if abs(width) > 1 else 'black')

plt.tight_layout()
plt.show()

# **Análisis de la Comparación de Tiempos Asignados vs Reales por Clasificación**

## 📊 **Diferencias entre Tiempos Asignados y Reales**
- En la mayoría de las clasificaciones, el **tiempo de entrega real** es mucho menor que el **tiempo asignado**.
- Esto indica que se está asignando más tiempo del necesario en varios casos.
- Sin embargo, hay excepciones como la categoría **E1**, donde el tiempo real (37.33 días) supera al asignado (30 días), lo que sugiere problemas en la planificación.

## ⚠️ **Casos de Mayor Diferencia**
- **Categoría 6**: Se asignaron **12 días**, pero el tiempo real fue solo **5 días**, lo que indica una posible sobreestimación.
- **Categoría F1**: Tiempo asignado de **12.60 días**, pero el real fue **8.80 días**, mostrando una tendencia similar.
- **Categoría II y III**: También tienen diferencias significativas, con tiempos reales muy por debajo de los asignados *(aunque estas se realizan en Sitio y el análisis es distinto).

## 🔍 **Categorías con Tiempos Ajustados**
- **M1 y M2** presentan diferencias menores entre los tiempos asignados y reales, lo que sugiere una planificación más precisa en estas clasificaciones.
- Esto podría deberse a una mejor estimación de los tiempos de mantenimiento en estas categorías.

## ✅ **Conclusiones y Recomendaciones**
1. **Optimización de los tiempos asignados**: Dado que en la mayoría de los casos los tiempos reales son menores, se puede ajustar la planificación para mejorar la eficiencia.
2. **Revisión de la categoría E1**: El tiempo real supera al asignado, lo que sugiere posibles retrasos o problemas en el proceso.
3. **Análisis de los factores que afectan el tiempo de entrega**: Revisar si hay elementos externos (falta de piezas, retrasos administrativos, etc.) que influyen en las variaciones entre los tiempos asignados y reales.

💡 **Próximo paso**: Implementar un análisis más detallado sobre los factores que generan las diferencias en cada categoría y ajustar la planificación en función de los datos obtenidos.


# ----------------------------------------------------------------

## Análisis de tendencias

In [ ]:
tiempo_gral = mass_sin_errores.loc[mass_sin_errores['service_location'] == 'LAB'][['report_number','classification', 'lab_received_date','assigned_time', 'delivery_date','delivery_time', ]].copy()

In [ ]:
# Agregar efectividad a la tabla
tiempo_gral['efectividad'] = (tiempo_gral['delivery_time']/tiempo_gral['assigned_time']).round(2)
tiempo_gral
salva = tiempo_gral.copy()

In [ ]:
# Renombrar columnas para evitar problemas 
tiempo_gral.rename(columns={'delivery_date ': 'delivery_date'}, inplace=True)


In [ ]:
# Convertir a datetime y crear columnas de periodo
#tiempo_gral['delivery_date'] = pd.to_datetime(tiempo_gral['delivery_date'])
tiempo_gral['semana'] = tiempo_gral['delivery_date'].dt.to_period('W-SUN')  # Semana que comienza en lunes
tiempo_gral['mes'] = tiempo_gral['delivery_date'].dt.to_period('M')

### Análisis de la tendencia de los tiempos por semana

In [ ]:
time_per_week = tiempo_gral.groupby(['semana'])['delivery_time'].mean().round(2).reset_index()
time_per_week['semana'] = time_per_week['semana'].dt.start_time
time_per_week['semana'] = time_per_week['semana'].dt.strftime('%d-%b')
time_per_week.rename(columns={'delivery_time': 'Tiempo Promedio'}, inplace=True)
time_per_week

In [ ]:
df = time_per_week.copy()

# Configuración de estilo profesional
sns.set_theme(style="whitegrid", context="talk")  # Estilo más moderno y tamaño de fuentes adecuado
plt.figure(figsize=(14, 7))  # Tamaño más grande para mejor visualización

# Paleta de colores personalizada
colors = sns.color_palette("husl", 2)

# Gráfico de línea principal
ax = sns.lineplot(
    data=df,
    x='semana',
    y='Tiempo Promedio',
    marker='o',
    markersize=10,
    linewidth=2.5,
    color=colors[0],
    markerfacecolor='white',
    markeredgewidth=2,
    label='Tiempo semanal'
)

# Personalización avanzada del gráfico
ax.set_title('Evolución Semanal del Tiempo Promedio de Entrega\nPrimer Trimestre 2024', 
             fontsize=16, pad=20, fontweight='bold')
ax.set_xlabel('Semana', fontsize=14, labelpad=15)
ax.set_ylabel('Días Promedio', fontsize=14, labelpad=15)

# Formateo de ejes
plt.xticks(rotation=50, ha='right', fontsize=12)
plt.yticks(fontsize=12)

# Línea de promedio general
mean_val = df['Tiempo Promedio'].mean()
plt.axhline(
    y=mean_val, 
    color=colors[1], 
    linestyle='--', 
    linewidth=2,
    alpha=0.8,
    label=f'Promedio general ({mean_val:.1f} días)'
)

# Etiquetas de datos mejoradas
for i, row in df.dropna().iterrows():
    ax.text(
        i, 
        row['Tiempo Promedio'] + 0.3,  # Ajuste de posición vertical
        f'{row["Tiempo Promedio"]:.1f}d', 
        ha='center', 
        va='bottom',
        fontsize=11,
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=2)
    )

# Elementos adicionales
sns.despine(left=True)  # Eliminar bordes para un look más limpio
plt.grid(axis='y', alpha=0.4)
plt.grid(axis='x', alpha=0.1)

# Leyenda mejorada
plt.legend(
    loc='upper right', 
    frameon=True, 
    framealpha=0.9,
    fontsize=12
)

# Ajuste final de layout
plt.tight_layout()

# Mostrar gráfico
plt.show()

# 📈 Análisis de Datos (Unidades Asumidas en Días)

## 📌 Métricas Clave
| **Indicador**       | **Valor**          | **Detalle**                         |
|----------------------|--------------------|--------------------------------------|
| Máximo Histórico     | 19.9 días          | Semana del 6 de enero                          |
| Mínimo Histórico     | 6.3 días           | Semana del 17 de febrero                            |
| Promedio Trimestral  | 10.2 días          |        |
| Casos >14 días       | 3 semanas          | Semanas 1, 2, 14 y 15               |
| Variabilidad         | 13.6 días          | Diferencia entre máximo y mínimo    |

---

## 🚨 Alertas Operativas
1. **Incremento Extremo**:  
   - Semana 1 (19.9d) supera en **216%** el mínimo histórico (6.3d)  
   - *Impacto*: Retraso equivalente a 3.3 semanas laborales estándar  

2. **Tendencia Ascendente Final**:  
   - Últimas 2 semanas: **14.6d** (36% sobre el promedio)  
   - *Patrón*: Aumento del 22% respecto a la semana 13 (12.0d)  

3. **Inconsistencia Temporal**:  
   - 15 semanas registradas (trimestre estándar: 13 semanas)  

---

## 🔄 Distribución Temporal
```python
semanas = [19.9, 16.6, 9.2, 8.9, 6.8, 7.5, 6.3, 7.1, 8.8, 7.1, 9.6, 11.7, 12.0, 14.6, 14.6]
distribucion = {
    '<7d': sum(1 for d in semanas if d < 7),      # 1 semana
    '7-10d': sum(1 for d in semanas if 7 <= d <=10),  # 7 semanas
    '10-14d': sum(1 for d in semanas if 10 < d <=14), # 4 semanas
    '>14d': sum(1 for d in semanas if d >14)          # 3 semanas
}

# 📈 Análisis de Datos 

## 📌 Métricas Clave
| **Indicador**       | **Valor**          | **Detalle**                         |
|----------------------|--------------------|--------------------------------------|
| Máximo Histórico     | 19.9 días          | Semana 2                            |
| Mínimo Histórico     | 6.3 días           | Semana 8                            |
| Promedio Trimestral  | 10.2 días          | ----------------------------        |
| Casos >14 días       | 3 semanas          | Semanas 1, 2 y 3                    |
| Variabilidad         | 13.6 días          | Diferencia entre máximo y mínimo    |

---

## 🚨 Alertas Operativas
1. **Incremento Extremo**:  
   - Semana 1 (19.9d) supera en **216%** el mínimo histórico (6.3d)  
   - *Impacto*: Retraso equivalente a 3.3 semanas laborales estándar  

2. **Tendencia Ascendente Final**:  
   - Últimas 2 semanas: **14.6d** (36% sobre el promedio)  
   - *Patrón*: Aumento del 22% respecto a la semana 13 (12.0d)  
 

---


### Análisis de la tendencia de los tiempos por meses

In [ ]:
time_per_month = tiempo_gral.groupby(['mes'])['delivery_time'].mean().round(2).reset_index()
time_per_month['mes'] = time_per_month['mes'].dt.start_time
time_per_month['mes'] = time_per_month['mes'].dt.strftime('%b')
time_per_month.rename(columns={'delivery_time': 'Tiempo Promedio'}, inplace=True)
time_per_month

In [ ]:
df_1 = time_per_month.copy()

# Configuración de estilo profesional
sns.set_theme(style="whitegrid", context="talk")  # Estilo más moderno y tamaño de fuentes adecuado
plt.figure(figsize=(14, 7))  # Tamaño más grande para mejor visualización

# Paleta de colores personalizada
colors = sns.color_palette("husl", 2)

# Gráfico de línea principal
ax = sns.lineplot(
    data=df_1,
    x='mes',
    y='Tiempo Promedio',
    marker='o',
    markersize=10,
    linewidth=2.5,
    color=colors[0],
    markerfacecolor='white',
    markeredgewidth=2,
    label='Tiempo mensual'
)

# Personalización avanzada del gráfico
ax.set_title('Evolución Mensual del Tiempo Promedio de Entrega\nPrimer Trimestre 2024', 
             fontsize=16, pad=20, fontweight='bold')
ax.set_xlabel('Mes', fontsize=14, labelpad=15)
ax.set_ylabel('Días Promedio', fontsize=14, labelpad=15)

# Formateo de ejes
plt.xticks(rotation=50, ha='right', fontsize=12)
plt.yticks(fontsize=12)

# Línea de promedio general
mean_val = df_1['Tiempo Promedio'].mean()
plt.axhline(
    y=mean_val, 
    color=colors[1], 
    linestyle='--', 
    linewidth=2,
    alpha=0.8,
    label=f'Promedio general ({mean_val:.1f} días)'
)

# Etiquetas de datos mejoradas
for i, row in df_1.dropna().iterrows():
    ax.text(
        i, 
        row['Tiempo Promedio'] + 0.3,  # Ajuste de posición vertical
        f'{row["Tiempo Promedio"]:.1f}d', 
        ha='center', 
        va='bottom',
        fontsize=11,
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=2)
    )

# Elementos adicionales
sns.despine(left=True)  # Eliminar bordes para un look más limpio
plt.grid(axis='y', alpha=0.4)
plt.grid(axis='x', alpha=0.1)

# Leyenda mejorada
plt.legend(
    loc='upper right', 
    frameon=True, 
    framealpha=0.9,
    fontsize=12
)

# Ajuste final de layout
plt.tight_layout()

# Mostrar gráfico
plt.show()

# 📌 INFORME DE ANÁLISIS: TIEMPOS DE ENTREGA (Q1 2024)

## 📊 **Hallazgos Clave**
- **Alta variabilidad**: Enero fue el peor mes (**14.8 días, +45% sobre el promedio**), mientras que marzo mejoró significativamente (**7.1 días, -30%**).
- **Enero domina la variabilidad**: Explica el **48% de la fluctuación total**.

## 🚨 **Riesgos Identificados**
- **Retraso crítico en enero**: Afecta tiempos de entrega en **1.5 semanas laborales**.
- **Tendencia irregular**: Reducción abrupta en marzo (se recomienda analizar las posibles causas).

## 🔍 **Recomendaciones**
1. **Investigar enero**: Determinar causas del pico (**14.8 días**).
2. **Replicar marzo**: Extender sus prácticas exitosas para estabilizar tiempos.
-